Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
# Import Statements
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
%load_ext autoreload
%autoreload 2

# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
print(train.shape, test.shape)

(4087, 3) (1022, 2)


In [3]:
train.head(4)

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1


In [4]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts(normalize=True)

1    0.704918
0    0.279178
2    0.015904
Name: ratingCategory, dtype: float64

In [5]:
# Read a few reviews from the "Excellent" category
pd.set_option('display.max_colwidth', 0)
train[train.ratingCategory == 0].sample(3)

,id,description,ratingCategory
2653,4682,"\nBunnahabhain’s peaty expression is steadily coming together. It’s been one of those drams that seems to need more time than many, but that’s true of Bunna’ in general, come to think of it. In fact, the peatiness is quite mild on the nose, adding some scent to the sandalwood elements and obvious raisined sweetness. There’s a slightly cheesy note in the background and a touch of sulphur on the palate. It’s not quite wholly integrated, but progressing well. (Fèis Ìle 2016, 833 bottles) £95",0
3301,3799,"\nThis bottling of Glenmorangie has been released to raise awareness of marine conservation. A proportion of the whisky has been finished in amontillado sherry butts. The nose offers honey, vanilla, peaches, toffee bonbons, wood lacquer, sherry, and a hint of peat. Soft and elegant on the palate, with fruity spice, nutty toffee, more sherry, and sweet smoke. Slightly smoky in the finish, with soft oak, citrus fruit, and aniseed. (Travel Retail only) £60",0
1102,4172,"\nAmrut ingeniously flavored an oloroso cask with wine and orange peel for 3 years before finishing this highly innovative whisky in it. A warmth and richness exudes, the citrus intensity of peel and orange oils develops the longer you resist temptation. Dried fruits, apricot, heather, ripe mango, triple sec, with a slight mustiness. Syrupy, soft orange pulp with zested limes makes it quite nippy. Finish of gum and wood notes after a spicy start. Whatever you do, drink it neat. (Europe, Canada, and Asia; 900 bottles) £75",0


In [6]:
# Read a few reviews from the "Poor" category
train[train.ratingCategory == 2].sample(3)

,id,description,ratingCategory
1846,5080,"\nMany moonshine products are centered around corn, but Freedom Moonshine mixes it up with a mashbill of 95% rye and 5% malted barley. Yeasty rye bread defines the nose with an undercurrent of white grape. The entry is soft, with yeasty rye, vanilla, and bran flakes. There’s some structure in the mid-palate with rye spice and black pepper but it needs more. The finish is a little flat, all pointing to the need for a slightly higher proof.",2
3890,5032,"\nCitrus peel, light maple syrup, and almonds, with emerging grape and vanilla. Somewhat elegant in nature, but the flavors do not especially complement each other. (Exclusive to Binny’s Beverage Depot.) \r\n",2
3299,5090,"\nThe wine cask anoints a gentle amber hue to this natural-colored whisky that has a nose redolent of wine gums. It has a firm texture with early promises of apple and blackberry, but then the flavors tumble off a cliff; a mouth-puckering bedlam of unsweetened rhubarb, cough medicine, licorice, and aniseed. A dash of water extinguishes many of the sour notes, but any redeeming qualities are weakened too, leaving it bland and saggy. Hard to love this one.",2


In [7]:
#before using 're' to clean
train['description'][0]

'\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only)\xa0A$133'

In [8]:
# import re library.
import re

# cleaning
def re_cleaning(X):
    
    X = X.apply(lambda x: re.sub('\d+\/\d+\/\d+', '', x))
    X = X.apply(lambda x: re.sub('^.*?([A-Z])', '\\1', x))
    X = X.apply(lambda x: re.sub(r'[^a-zA-Z ^0-9]', '', x))
    X = X.apply(lambda x: x.lower())
    
    return X
    
train['description'] = re_cleaning(train['description'])

In [9]:
#after using 're' to clean
train['description'][0]

'sometimes when whisky is batched a few leftover barrels are returned to the warehouse canadian club recently pulled and vatted several of these from the 1970s acetone granny smith apples and freshcut white cedar showcase this long age complex and spicy yet reserved this dram is ripe with strawberries canned pears cloves pepper and faint flowers then slightly pulling oak tannins distinct elegant and remarkably vibrant this ancient canadian club is anything but tired australia onlya133'

In [10]:
train.head(3)

,id,description,ratingCategory
0,1321,sometimes when whisky is batched a few leftover barrels are returned to the warehouse canadian club recently pulled and vatted several of these from the 1970s acetone granny smith apples and freshcut white cedar showcase this long age complex and spicy yet reserved this dram is ripe with strawberries canned pears cloves pepper and faint flowers then slightly pulling oak tannins distinct elegant and remarkably vibrant this ancient canadian club is anything but tired australia onlya133,1
1,3861,an uncommon exclusive bottling of a 6 year old cask strength malt light gold in color the nose is vegetal more peat bog than peat smoke with an undercurrent of pastry cream and rose its an odd combination of aromas the entry is flavorful and inviting with smoked pineapple clove and rose peak smoke arrives in full force in the midpalate which drops the sweet and becomes spicy the finish is mostly smoke but with a pleasant minty coolness wyoming only,0
2,655,this release is a port version of amruts intermediate sherry a sort of port pipe sandwich the spirit is matured in both unused casks and bourbon casks then spends a few months in port pipes and then returns to bourbon casks the result is a pink floyd show of a whisky vibrant colorful complex and nearly too much a blackcurrant and wispy smoky nose gives way to an intense and bittersweet mix of chili blackcurrant oak damson dark chocolate and peat astounding,1


### Define Pipeline Components

In [11]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [12]:
data = train
target = train['ratingCategory']

In [13]:
#Checking for the best number of workers
N_JOBS = [None, 1, 2, 3, 4, -1]
import time
for n_jobs in N_JOBS:
        start = time.perf_counter()
        print('n_jobs = {}, perf_counter = {}'.format(n_jobs, time.perf_counter() - start))

n_jobs = None, perf_counter = 6.629999802498787e-07
n_jobs = 1, perf_counter = 8.089999994354002e-07
n_jobs = 2, perf_counter = 4.4300000467956124e-07
n_jobs = 3, perf_counter = 3.7500001326407073e-07
n_jobs = 4, perf_counter = 3.9799999740353087e-07
n_jobs = -1, perf_counter = 2.3454000000810993e-05


In [14]:
%%time
parameters = {
    'vect__max_df': (.5, 0.7, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (250, 500, 1000, 1200),
    'clf__n_estimators':( 5, 6, 7, 8, 9, 10, 25, 50, 100),
    'clf__max_depth':(12, 15, 20, 25)
}

grid_search = GridSearchCV(pipe,parameters, cv=2, n_jobs=4, verbose=7)
grid_search.fit(data.description, target)
print(grid_search.best_params_)

Fitting 2 folds for each of 864 candidates, totalling 1728 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    6.6s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:   16.4s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:   30.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   49.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 384 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done 504 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  4.0min
[Parallel(n_jobs=4)]: Done 960 tasks      | elapsed:  4.9min
[Parallel(n_jobs=4)]: Done 1144 tasks      | elapsed:  5.8min
[Parallel(n_jobs=4)]: Done 1344 tasks      | elapsed:  7.1min
[Parallel(n_jobs=4)]: Done 1560 tasks      | elapsed:  8.3min
[Parallel(n_jobs=4)]: Done 1728 out of 1728 | elapsed:  9.7min fi

In [15]:
print(grid_search.best_score_)

0.721310919918427


In [ ]:
# vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
# rfc = RandomForestClassifier()
# logreg = LogisticRegression(C=1e5)

# pipe = Pipeline([('vect', vect), ('clf', logreg)])

# #max_iter=100, multi_class='auto'

# parameters = {
#     'vect__max_df': (.4, .5, 0.7, 1.0),
#     'vect__min_df': (.02, .05),
#     'vect__max_features': (250, 500, 1000, 1200),
#     'clf__max_iter':(5,6, 7, 8, 9, 10, 999999)
#     'clf__max_depth':(8, 9, 10, 12, 20)
# }

# grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=5)
# grid_search.fit(data.description, target)

In [ ]:
# print(grid_search.best_score_)
# print(grid_search.best_params_)

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [27]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [28]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [29]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [30]:
subNumber = 3

In [31]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [ ]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [ ]:
lsi = ...
vect = ...
clf = ...

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(..., ...)

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [ ]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
# Continue Word Embedding Work Here


### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?